Author: Yiyao Jin

In [1]:
import pandas as pd

In [2]:
import pysam
import os

In [3]:
refSeq = pd.read_table('./refseq.genes.ncbi37')

In [4]:
refSeq.head(10)

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames
0,0,NM_032291,chr1,+,66999824,67210768,67000041,67208778,25,"66999824,67091529,67098752,67101626,67105459,6...","67000051,67091593,67098777,67101698,67105516,6...",0,SGIP1,cmpl,cmpl,"0,1,2,0,0,0,1,0,0,0,1,2,1,1,1,1,0,1,1,2,2,0,2,..."
1,1,NM_052998,chr1,+,33546713,33585995,33547850,33585783,12,"33546713,33546988,33547201,33547778,33549554,3...","33546895,33547109,33547413,33547955,33549728,3...",0,ADC,cmpl,cmpl,"-1,-1,-1,0,0,0,2,2,0,1,0,2,"
2,1,NM_001145278,chr1,+,16767166,16786584,16767256,16785385,8,"16767166,16770126,16774364,16774554,16775587,1...","16767270,16770227,16774469,16774636,16775696,1...",0,NECAP2,cmpl,cmpl,"0,2,1,1,2,0,1,2,"
3,1,NM_001145277,chr1,+,16767166,16786584,16767256,16785491,7,"16767166,16770126,16774364,16774554,16775587,1...","16767348,16770227,16774469,16774636,16775696,1...",0,NECAP2,cmpl,cmpl,"0,2,1,1,2,0,1,"
4,1,NM_001080397,chr1,+,8384389,8404227,8384389,8404073,8,"8384389,8385357,8385877,8390268,8395496,839787...","8384786,8385450,8386102,8390996,8395650,839805...",0,SLC45A1,cmpl,cmpl,"0,1,1,1,0,1,1,0,"
5,1,NM_018090,chr1,+,16767166,16786584,16767256,16785385,8,"16767166,16770126,16774364,16774554,16775587,1...","16767348,16770227,16774469,16774636,16775696,1...",0,NECAP2,cmpl,cmpl,"0,2,1,1,2,0,1,2,"
6,1,NM_013943,chr1,+,25071759,25170815,25072044,25167428,6,"25071759,25124232,25140584,25153500,25166350,2...","25072116,25124342,25140710,25153607,25166532,2...",0,CLIC4,cmpl,cmpl,"0,0,2,2,1,0,"
7,1,NM_032785,chr1,-,48998526,50489626,48999844,50489468,14,"48998526,49000561,49005313,49052675,49056504,4...","48999965,49000588,49005410,49052838,49056657,4...",0,AGBL4,cmpl,cmpl,"2,2,1,0,0,2,1,1,0,2,0,1,1,0,"
8,2,NM_001195684,chr1,-,92145899,92371559,92149295,92327088,18,"92145899,92161228,92163645,92174219,92177799,9...","92149414,92161336,92163687,92174340,92178099,9...",0,TGFBR3,cmpl,cmpl,"1,1,1,0,0,0,0,0,1,0,2,1,0,0,1,0,-1,-1,"
9,2,NM_001195683,chr1,-,92145899,92351836,92149295,92327088,17,"92145899,92161228,92163645,92174219,92177799,9...","92149414,92161336,92163687,92174340,92178099,9...",0,TGFBR3,cmpl,cmpl,"1,1,1,0,0,0,0,0,1,0,2,1,0,0,1,0,-1,"


In [5]:
codonMap = {"TTT":"F", "TTC":"F", "TTA":"L", "TTG":"L",
    "TCT":"S", "TCC":"S", "TCA":"S", "TCG":"S",
    "TAT":"Y", "TAC":"Y", "TAA":"STOP", "TAG":"STOP",
    "TGT":"C", "TGC":"C", "TGA":"STOP", "TGG":"W",
    "CTT":"L", "CTC":"L", "CTA":"L", "CTG":"L",
    "CCT":"P", "CCC":"P", "CCA":"P", "CCG":"P",
    "CAT":"H", "CAC":"H", "CAA":"Q", "CAG":"Q",
    "CGT":"R", "CGC":"R", "CGA":"R", "CGG":"R",
    "ATT":"I", "ATC":"I", "ATA":"I", "ATG":"M",
    "ACT":"T", "ACC":"T", "ACA":"T", "ACG":"T",
    "AAT":"N", "AAC":"N", "AAA":"K", "AAG":"K",
    "AGT":"S", "AGC":"S", "AGA":"R", "AGG":"R",
    "GTT":"V", "GTC":"V", "GTA":"V", "GTG":"V",
    "GCT":"A", "GCC":"A", "GCA":"A", "GCG":"A",
    "GAT":"D", "GAC":"D", "GAA":"E", "GAG":"E",
    "GGT":"G", "GGC":"G", "GGA":"G", "GGG":"G",}

In [6]:
def ReverseComplement(Text):
    reversedComplement = ""
    for x in range(0,len(Text)):
        if Text[x] == "A":
            curr = "T"
        elif Text[x] == "T":
            curr = "A"
        elif Text[x] == "G":
            curr = "C"
        else:
            curr = "G"
        reversedComplement = curr + reversedComplement
    return reversedComplement

In [7]:
def ExtractExonSeqs(data):
    exonSeqs = [""]*len(data)
    extractSeq = pysam.FastaFile('./chromFa/' + 'chr1' +'.fa')
    
    for i in range(len(data)):
        start = data['exonStarts'][i][:-1].split(',')
        end = data['exonEnds'][i][:-1].split(',')
        
        if i != 0 and data['chrom'][i] != data['chrom'][i-1]:
            extractSeq = pysam.FastaFile('./chromFa/' + data['chrom'][i] +'.fa')
        s = ""
        readFrame = data['exonFrames'][i][:-1].split(',')
        
        for j in range(len(start)):
            if readFrame[j] == str(-1):
                continue
            s += extractSeq.fetch("", int(start[j]), int(end[j]), data['chrom'][i])
        
        exonSeqs[i] = s
        
        if data['strand'][i] == '-':
            exonSeqs[i] = ReverseComplement(s)
    
    return exonSeqs

In [8]:
def Calculatelength(data):
    length = [0]*len(data)
    for i in range(len(data)):
        start = data['exonStarts'][i][:-1].split(',')
        end = data['exonEnds'][i][:-1].split(',')
        length[i] = [0]*len(start)
        for j in range(len(start)):
            length[i][j] = int(end[j]) - int(start[j])
        if data['strand'][i] == '-':
            length[i] = list(reversed(length[i]))
    return length

In [9]:
exonLength = Calculatelength(refSeq)

In [10]:
exonSeqsO = ExtractExonSeqs(refSeq)

In [11]:
def Translation(codonmap, seqs, data):
    threeReadingFrame = []
    # 3 reading frame

    for i in range(len(seqs)):
        start = data['cdsStart'][i] - data['txStart'][i]
        end = len(seqs[i]) + data['cdsEnd'][i] - data['txEnd'][i]
        aa = []
        for x in range(0,3):
            aminoAcid = ""
            for j in range(start+x, end-2, 3):
                codon = codonmap[seqs[i][j:j+3].upper()]
                if codon == "STOP":
                    break
                aminoAcid += codon
            aa.append(aminoAcid)
        threeReadingFrame.append(aa)
        
    return threeReadingFrame

In [12]:
transcript = Translation(codonMap, exonSeqsO, refSeq)    

In [13]:
with open("transcript.txt", "w") as file_out:
    file_out.write('ReadingFrame1 ReadingFrame2 ReadingFrame3'+ '\n')
    for x in range(len(transcript)):
        for y in range(len(transcript[x])):
            file_out.write(transcript[x][y] + '-')
        file_out.write('\n')

In [14]:
def ExtractExonSeqsFrag(data):
    exonSeqs = [""]*len(data)
    extractSeq = pysam.FastaFile('./chromFa/' + 'chr1' +'.fa')
    
    for i in range(len(data)):
        exonNum = data['exonCount'][i]
        start = data['exonStarts'][i][:-1].split(',')
        end = data['exonEnds'][i][:-1].split(',')
        
        # update chromosome if different from the last one
        if i != 0 and data['chrom'][i] != data['chrom'][i-1]:
            extractSeq = pysam.FastaFile('./chromFa/' + data['chrom'][i] +'.fa')
        s = ""
        exons = [0]*exonNum 
        readFrame = data['exonFrames'][i][:-1].split(',')
        
        #for each line, extract all the exons
        for j in range(exonNum):
            if readFrame[j] == str(-1):
                continue
            s = extractSeq.fetch("", int(start[j]), int(end[j]), data['chrom'][i])
            if data['strand'][i] == '-':
                exons[exonNum-1-j]=ReverseComplement(s)
            else:  
                exons[j]=s
   
        exonSeqs[i] = exons
    return exonSeqs

In [15]:
exonSeqsF = ExtractExonSeqsFrag(refSeq)

In [16]:
testSeq = exonSeqsF

In [17]:
def checkStopCodon(sequence,table):
    for x in range(0, len(sequence),3):
        codon = table[sequence[x:x+3].upper()]
        if codon == "STOP":
            return False
    return True

In [18]:
def checkRangeInsert1_before(data,seqs,i,j,k,nucleotide):
    lengthSeq = len(seqs[i][j])
    readFrame = data['exonFrames'][i][:-1].split(',')[j]
    # insert at beginning
    
    #check delete previous exon to rescue
    # insert after each index
    # add boundary condition for k >= lengthSeq-2
    if k < lengthSeq-1:
        off = k%3
        #position mod 3
        if off == 0:
        # check different reading frames
            if readFrame == '0':
                return seqs[i][j-2][-2:]+seqs[i][j][:k+1]+nucleotide+seqs[i][j][k+1:k+3]
            elif readFrame == '1':
                return seqs[i][j-2][-1]+seqs[i][j][:k+1]+nucleotide
            else:
                return seqs[i][j][:k+1]+nucleotide+seqs[i][j][k+1]
        elif off == 1:
            if readFrame == '0':
                return seqs[i][j-2][-2:]+seqs[i][j][:k+1]+nucleotide+seqs[i][j][k+1]
            elif readFrame == '1':
                return seqs[i][j-2][-1]+seqs[i][j][:k+1]+nucleotide+seqs[i][j][k+1:k+3]
            else:
                return seqs[i][j][:k+1]+nucleotide
        else:
            if readFrame == '0':
                return seqs[i][j-2][-2:]+seqs[i][j][:k+1]+nucleotide
            elif readFrame == '1':
                return seqs[i][j-2][-1]+seqs[i][j][:k+1]+nucleotide+seqs[i][j][k+1]
            else:
                return seqs[i][j][:k+1]+nucleotide+seqs[i][j][k+1:k+3]

In [19]:
def checkRangeInsert2_before(data,seqs,i,j,k,nucleotide):
    lengthSeq = len(seqs[i][j])
    readFrame = data['exonFrames'][i][:-1].split(',')[j]
    # insert at beginning
    
    #check delete previous exon to rescue
    # insert after each index
    # add boundary condition for k >= lengthSeq-2
    if k < lengthSeq-2:
        off = k%3
        #position mod 3
        if off == 0:
        # check different reading frames
            if readFrame == '0':
                return seqs[i][j-2][-1]+seqs[i][j][:k+1]+nucleotide+seqs[i][j][k+1:k+3]
            elif readFrame == '2':
                return seqs[i][j][:k+1]+nucleotide
            else:
                return seqs[i][j-2][-2:]+seqs[i][j][:k+1]+nucleotide+seqs[i][j][k+1]
        elif off == 1:
            if readFrame == '0':
                return seqs[i][j-2][-1:]+seqs[i][j][:k+1]+nucleotide+seqs[i][j][k+1]
            elif readFrame == '2':
                return seqs[i][j][:k+1]+nucleotide+seqs[i][j][k+1:k+3]
            else:
                return seqs[i][j-2][-2:]+seqs[i][j][:k+1]+nucleotide
        else:
            if readFrame == '0':
                return seqs[i][j-2][-1:]+seqs[i][j][:k+1]+nucleotide
            elif readFrame == '2':
                return seqs[i][j][:k+1]+seqs[i][j][:k+1]+nucleotide+seqs[i][j][k+1]
            else:
                return seqs[i][j-2][-2:]+nucleotide+seqs[i][j][k+1:k+3]

In [20]:
checkRangeInsert2_before(refSeq,testSeq,1,5,3,'GC')

'GGCAGGCAG'

In [21]:
def checkRangeInsert_after(data,seqs,i,j,k,nucleotide):
    lengthSeq = len(seqs[i][j])
    readFrame = data['exonFrames'][i][:-1].split(',')[j]
    # insert at beginning
    
    
    # insert after each index
    if k < lengthSeq-2:
        off = k%3
        result = ''
        #position mod 3
        if off == 0:
        # check different reading frames
            if readFrame == '0':
                result = seqs[i][j][k]+nucleotide+seqs[i][j][k+1:]
            elif readFrame == '2':
                result = nucleotide+seqs[i][j][k+1:]
            else:
                result = seqs[i][j][k-1:k+1]+nucleotide+seqs[i][j][k+1:]
        elif off == 1:
            if readFrame == '0':
                result = seqs[i][j][k-1:k+1]+nucleotide+seqs[i][j][k+1:]
            elif readFrame == '2':
                result = seqs[i][j][k]+nucleotide+seqs[i][j][k+1:]
            else:
                result = nucleotide+seqs[i][j][k+1:]
        else:
            if readFrame == '0':
                result = nucleotide+seqs[i][j][k+1:]
            elif readFrame == '2':
                result = seqs[i][j][k-1:k+1]+nucleotide+seqs[i][j][k+1:]
            else:
                result = seqs[i][j][k]+nucleotide+seqs[i][j][k+1:]
        if len(result)%3 == 0:
            return result
        elif len(result)%3 == 1:
            return result + seqs[i][j+2][:2]
        else:
            return result + seqs[i][j+2][:1]

In [22]:
len(checkRangeInsert_after(refSeq,testSeq,1,5,3,'G'))

171

In [23]:
def checkRangedelete2_before(data,seqs,i,j,k):
    exonSeq = len(seqs[i][j])
    readFrame = data['exonFrames'][i][:-1].split(',')[j]
    # insert at beginning
    
    #check delete previous exon to rescue
    # insert after each index
    # add boundary condition for k >= lengthSeq-2
    if k < exonSeq-1:
        off = k%3
        #position mod 3
        if off == 0:
        # check different reading frames
            if readFrame == '0':
                #print(i,j,k)
                return seqs[i][j-2][-2:]+seqs[i][j][:k]+seqs[i][j][k+2]
            elif readFrame == '2':
                return seqs[i][j-2][-1]+seqs[i][j][:k]+seqs[i][j][k+2:k+4]
            else:
                return seqs[i][j][:k]
        elif off == 1:
            if readFrame == '0':
                return seqs[i][j-2][-2:]+seqs[i][j][:k]
            elif readFrame == '2':
                return seqs[i][j-2][-1]+seqs[i][j][:k]+seqs[i][j][k+2]
            else:
                return seqs[i][j][:k]+seqs[i][j][k+2:k+4]
        else:
            if readFrame == '0':
                return seqs[i][j-2][-2:]+seqs[i][j][:k]+seqs[i][j][k+2:k+4]
            elif readFrame == '2':
                return seqs[i][j-2][-1]+seqs[i][j][:k]
            else:
                return seqs[i][j][:k]+seqs[i][j][k+2]

In [24]:
checkRangedelete2_before(refSeq,testSeq,7,3,92)

'TGAGGGTCATTTTCAACATTGTTAACTTCAGTAAAACCAAGAGTCTCTATAGAGATGGGATGGCCCCTATGGTGAAATCTACCAGCAGACCAAAG'

In [25]:
def checkRangedelete1_before(data,seqs,i,j,k):
    lengthSeq = len(seqs[i][j])
    readFrame = data['exonFrames'][i][:-1].split(',')[j]
    # insert at beginning
    
    #check delete previous exon to rescue
    # insert after each index
    # add boundary condition for k >= lengthSeq-2
    if k < lengthSeq:
        off = k%3
        #position mod 3
        if off == 0:
        # check different reading frames
            if readFrame == '0':
                return seqs[i][j-2][-1]+seqs[i][j][:k]+seqs[i][j][k+1:k+3]
            elif readFrame == '2':
                return seqs[i][j][:k]
            else:
                return seqs[i][j-2][-2:]+seqs[i][j][:k]+seqs[i][j][k+1]
        elif off == 1:
            if readFrame == '0':
                return seqs[i][j-2][-1]+seqs[i][j][:k]+seqs[i][j][k+1]
            elif readFrame == '2':
                return seqs[i][j][:k]+seqs[i][j][k+1:k+3]
            else:
                return seqs[i][j-2][-2:]+seqs[i][j][:k]
        else:
            if readFrame == '0':
                return seqs[i][j-2][-1]+seqs[i][j][:k]
            elif readFrame == '2':
                return seqs[i][j][:k]+seqs[i][j][k+1]
            else:
                return seqs[i][j-2][-2:]+seqs[i][j][:k]+seqs[i][j][k+1:k+3]

In [26]:
checkRangedelete1_before(refSeq,testSeq,7,3,92)

'GAGGGTCATTTTCAACATTGTTAACTTCAGTAAAACCAAGAGTCTCTATAGAGATGGGATGGCCCCTATGGTGAAATCTACCAGCAGACCAAA'

In [27]:
refSeq['exonFrames'][1][:-1].split(',')[6]

'2'

In [28]:
def checkRangeDelete_after(data,seqs,i,j,k,num):
    lengthSeq = len(seqs[i][j])
    readFrame = data['exonFrames'][i][:-1].split(',')[j]
    # insert at beginning
    
    
    # insert after each index
    if k < lengthSeq-2:
        off = k%3
        result = ''
        #position mod 3
        if off == 0:
        # check different reading frames
            if readFrame == '0':
                result = seqs[i][j][k+num:]
            elif readFrame == '2':
                result = seqs[i][j][k-2:k]+seqs[i][j][k+num:]
            else:
                result = seqs[i][j][k-1]+seqs[i][j][k+num:]
        elif off == 1:
            if readFrame == '0':
                result = seqs[i][j][k-1]+seqs[i][j][k+num:]
            elif readFrame == '2':
                result = seqs[i][j][k+num:]
            else:
                result = seqs[i][j][k-2:k]+seqs[i][j][k+num:]
        else:
            if readFrame == '0':
                result = seqs[i][j][k-2:k]+seqs[i][j][k+num:]
            elif readFrame == '2':
                result = seqs[i][j][k-1]+seqs[i][j][k+num:]
            else:
                result = seqs[i][j][k+num:]
        if len(result)%3 == 0:
            return result
        elif len(result)%3 == 1:
            return result + seqs[i][j+2][:2]
        else:
            return result + seqs[i][j+2][:1]

In [29]:
len(checkRangeDelete_after(refSeq,testSeq,1,5,3,1))

171

In [30]:
def findcds(seqs):
    cdsRegion = [0]*len(seqs)
    for i in range(len(seqs)):
        for j in range(len(seqs[i])):
            if seqs[i][j]!= 0:
                cdsRegion[i] = [0]*2
                cdsRegion[i][0] = j
                break
        for j in range(len(seqs[i])):
            if seqs[i][len(seqs[i])-1-j]!= 0:
                cdsRegion[i][1] = len(seqs[i])-1-j
                break
    return cdsRegion

In [31]:
def frameshift(data, seqs, i, j, k, cmap, PoN, exonL, posRes):

    # frameshift +1
    if exonL[i][j+PoN]%3 == 1:
        if PoN == -1:
            #check new possible stop code, insert 1
            for m in {'A', 'C', 'G', 'T'}:
                checkRangeI = checkRangeInsert1_before(data,seqs,i,j,k,m)
                if checkStopCodon(checkRangeI,cmap):
                    posRes[0] = m
            #Or delete 2 nucleotide
            checkRangeD = checkRangedelete2_before(data,seqs,i,j,k)
            #print(i,j,k)
            if checkStopCodon(checkRangeD,cmap):
                posRes[1] = 'd2'
        else:
            for m in {'A', 'C', 'G', 'T'}:
                checkRangeI = checkRangeInsert_after(data,seqs,i,j,k,m)
                if checkStopCodon(checkRangeI,cmap):
                    posRes[2] = m
            #Or delete 2 nucleotide
            checkRangeD = checkRangeDelete_after(data,seqs,i,j,k,2)
            if checkStopCodon(checkRangeD,cmap):
                posRes[3] = 'd2'
    # frameshift +2
    elif exonL[i][j+PoN]%3 == 2:
        if PoN == -1:
            for m in {'AA', 'AC', 'AG', 'AT', 
                      'CA', 'CC', 'CG', 'CT', 
                      'GA', 'GC', 'GG', 'GT',
                      'TA', 'TC', 'TG', 'TT'}:
                checkRangeI = checkRangeInsert2_before(data,seqs,i,j,k,m)
                if checkStopCodon(checkRangeI,cmap):
                    posRes[4] = m
            checkRangeD = checkRangedelete1_before(data,seqs,i,j,k)
            if checkStopCodon(checkRangeD,cmap):
                posRes[5] = 'd1'
        else:
            for m in {'AA', 'AC', 'AG', 'AT', 
                      'CA', 'CC', 'CG', 'CT', 
                      'GA', 'GC', 'GG', 'GT',
                      'TA', 'TC', 'TG', 'TT'}:
                checkRangeI = checkRangeInsert_after(data,seqs,i,j,k,m)
                if checkStopCodon(checkRangeI,cmap):
                    posRes[6] = m
            checkRangeD = checkRangeDelete_after(data,seqs,i,j,k,1)
            if checkStopCodon(checkRangeD,cmap):
                posRes[7] = 'd1'
    return posRes

In [32]:
def PositionRescue(data, seqs, i, exonS, exonD, cmap, exonL):
    readFrame = data['exonFrames'][i][:-1].split(',')
    exonNum = data['exonCount'][i]
    pR = [0]*exonNum
    
    # delete the previous exon to rescue
    for j in range(exonS+2,exonD+1):  
        pR[j] = [0]*exonL[i][j]
        # iterate positions
        # insert after each position, delete that position
        for h in range(exonL[i][j]-3):
            #print(i,j,h)
            pR[j][h] = frameshift(data, seqs, i, j, h, cmap, -1, exonL, [0]*8)
            
            
    # delete the next exon to rescue
    for j in range(exonS,exonD-1):  
        pR[j] = [0]*exonL[i][j]
        # insert after each position, delete that position
        for h in range(exonL[i][j]-3):
            pR[j][h] = frameshift(data, seqs, i, j, h, cmap, 1, exonL, [0]*8)
               
    return pR


In [33]:
len(testSeq[7][4])

217

In [34]:
findcds(testSeq)[0]

[0, 24]

In [36]:
PositionRescue(refSeq, testSeq, 0, 0, 24, codonMap, exonLength)[0][0] != [0]*8

False

In [50]:
def Rescue(data, seqs, cmap):
    with open("Rescue.txt", "w") as file_out:
        file_out.write('Gene Exon Position Frameshift Exon_skipped Comments'+'\n')
        possibleRescue = [0]*len(data)
        exonLen = Calculatelength(data)
        cdsR = findcds(seqs)

        for i in range(len(seqs)):
            if cdsR[i] == 0:
                continue
            s = cdsR[i][0]
            e = cdsR[i][1]
            possibleRescue[i] = PositionRescue(data, seqs, i, s ,e, cmap, exonLen)
            for j in range(data['exonCount'][i]):
                if possibleRescue[i][j] != 0:
                    for k in range(exonLen[i][j]):
                        if possibleRescue[i][j][k] != 0 and possibleRescue[i][j][k] != [0]*8:
                            #print(possibleRescue[i][j][k])
                            file_out.write(data['name'][i]+' '+str(j)+' '+str(k)+' ')
                            if possibleRescue[i][j][k][0] or possibleRescue[i][j][k][1] or possibleRescue[i][j][k][2] or possibleRescue[i][j][k][3]!= 0:
                                file_out.write('+1'+' ')
                            else:
                                file_out.write('+2'+' ')
                            if possibleRescue[i][j][k][0] or possibleRescue[i][j][k][1] or possibleRescue[i][j][k][4] or possibleRescue[i][j][k][5]!= 0:
                                file_out.write(str(k-1)+' ')
                            else:
                                file_out.write(str(k+1)+' ')
                            for x in range(8):
                                if possibleRescue[i][j][k][x] != 0:
                                    file_out.write(str(possibleRescue[i][j][k][x])+' ')
                            file_out.write('\n')

In [52]:
Rescue(refSeq, testSeq, codonMap)

KeyboardInterrupt: 

In [ ]:
len(testSeq[6][5])

In [ ]:
def possibleRescue(data,seqs):
    total = 0
    exonLen = Calculatelength(data)
    cdsR = findcds(seqs)
    for i in range(len(exonLen)):
        if cdsR[i] != 0:
            for j in range(cdsR[i][0]+1,cdsR[i][1]):
                if exonLen[i][j-1]%3 != 0 or exonLen[i][j+1]%3 != 0:
                    total += 1
            if cdsR[i][0]+1< len(exonLen[i]) and exonLen[i][cdsR[i][0]+1]%3 != 0:
                total += 1
            if cdsR[i][1]-1> cdsR[i][0] and exonLen[i][cdsR[i][1]-1]%3 != 0:
                total += 1
    total1= 0
    for i in range(len(exonLen)):
        total1 += data['exonCount'][i]
    return total/total1

In [ ]:
possibleRescue(refSeq,testSeq)

In [ ]:
flag = False
                        for l in range(8):
                            if possibleRescue[i][j][k][l] != 0:
                                flag = True
                                break
                        if flag: